# 🚀 Module 2: Model Training and Experiment Tracking

In this module, we will:
1. Load the Processed Dataset
2. Define Features and Target Variable
3. Train a Machine Learning Model and Evaluate it's Performance
4. Track Experiments and Parameters using **MLflow**
6. Register the Trained Model with the Best Performanc

Make sure MLflow is installed in your environment:

```bash
pip install mlflow
```

In [21]:
# Install requirements
!pip install -r requirements.txt


[notice] A new release of pip is available: 24.2 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


## 📦 Import Required Libraries

Before we proceed with training and tracking our machine learning model, we need to import the necessary libraries.


In [1]:
# Import necessary modules
import os
import random

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score

import pandas as pd
import numpy as np

import mlflow
import mlflow.sklearn
from mlflow.tracking import MlflowClient

## 📥 Load the Processed Dataset

We'll start by loading the processed dataset for January 2011, which was prepared in the data exploration phase.

This dataset contains cleaned and feature-engineered data and will be used as the reference dataset for drift and performance comparison.

We use `pandas` to read the CSV file and inspect the first few rows.

In [2]:
# Load the training data
data_path = "./data/processed/"

# Read both CSV files
data_01 = pd.read_csv(data_path + 'data_2011_01.csv')
data_02 = pd.read_csv(data_path + 'data_2011_02.csv')
# data_03 = pd.read_csv(data_path + 'data_2011_03.csv')

# Concatenate the datasets
# input_data_df = pd.concat([data_01, data_02, data_03], ignore_index=True)
input_data_df = pd.concat([data_01, data_02], ignore_index=True)

input_data_df.head()

,dteday,instant,season,year,month,hour,holiday,weekday,workingday,weathersit,temp,atemp,humidity,windspeed,casual,registered,count
0,2011-01-01,1,1,0,1,0,0,6,0,1,0.24,0.2879,0.81,0.0,3,13,16
1,2011-01-01,2,1,0,1,1,0,6,0,1,0.22,0.2727,0.80,0.0,8,32,40
2,2011-01-01,3,1,0,1,2,0,6,0,1,0.22,0.2727,0.80,0.0,5,27,32
3,2011-01-01,4,1,0,1,3,0,6,0,1,0.24,0.2879,0.75,0.0,3,10,13
4,2011-01-01,5,1,0,1,4,0,6,0,1,0.24,0.2879,0.75,0.0,0,1,1


## 🔧 Feature Selection
In this section, we define the features used for model training by separating them into numerical and categorical variables:

- Numerical Features:

        ['temp', 'atemp', 'humidity', 'windspeed', 'hour', 'weekday']

These features represent continuous or ordered values, such as temperature and time-related attributes.

- Categorical Features:

        ['season', 'holiday', 'workingday', 'weathersit']

These features represent discrete categories or binary flags that indicate specific conditions or periods.

This separation is essential for preprocessing, allowing us to apply appropriate transformations to each feature type.

In [3]:
numerical_features=['temp', 'atemp', 'humidity', 'windspeed', 'hour', 'weekday']
categorical_features=['season', 'holiday', 'workingday', 'weathersit']

## ✂️ Define Features and Target Variable

We prepare the dataset for training by selecting input features and the target variable:

- Input Features (X_input): A combination of both numerical and categorical features defined earlier.

- Target Variable (y_input): The target for prediction is the count column, representing the total number of bike rentals.

Next, we perform a train-test split using an 70/30 ratio to evaluate model performance on unseen data. The random_state=42 ensures reproducibility of results.

In [4]:
X_input = input_data_df[numerical_features + categorical_features]
y_input = input_data_df["count"]

# X_train.head()

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X_input, y_input, test_size=0.3, random_state=42)

Displaying the first few rows of the **train feature set** to verify the structure and content:

In [5]:
X_train.head()

,temp,atemp,humidity,windspeed,hour,weekday,season,holiday,workingday,weathersit
138,0.20,0.1970,0.64,0.1940,0,5,1,0,1,2
381,0.16,0.1515,0.47,0.2239,9,1,1,1,0,2
292,0.24,0.2121,0.38,0.2985,15,4,1,0,1,1
1089,0.46,0.4545,0.67,0.2537,3,5,1,0,1,1
892,0.20,0.1970,0.37,0.2537,16,3,1,0,1,2


Displaying the first few rows of the **test feature set** to verify the structure and content:

In [6]:
X_test.head()

,temp,atemp,humidity,windspeed,hour,weekday,season,holiday,workingday,weathersit
899,0.16,0.1515,0.80,0.1940,23,3,1,0,1,3
1063,0.34,0.3182,0.53,0.2239,1,4,1,0,1,1
1255,0.52,0.5000,0.77,0.4478,11,5,1,0,1,1
298,0.14,0.1212,0.46,0.2985,21,4,1,0,1,1
237,0.16,0.1818,0.55,0.1343,6,2,1,0,1,2


## 🤖 Train a Regression Model

We train a Random Forest Regressor to predict bike rental counts using the training dataset:

``n_estimators``:  Number of decision trees that the model builds (i.e 50, 100, 200)

``max_depth``: Maximum depth of each tree (i.e 2, 6, 10, 15)

``random_state``: Ensures reproducibility of results by fixing the random seed. (i.e 42)

After fitting the model on X_train and y_train, we generate predictions on the test set (X_test) and store them in y_pred. These predictions will be evaluated to measure model performance.

In [93]:
# Define and train model
n_estimators = 100
max_depth=10
random_state = 42

model_randomforest = RandomForestRegressor(
    n_estimators=n_estimators, 
    random_state=random_state,
    max_depth=max_depth
    )
model_randomforest.fit(X_train, y_train)

# Predict on test set
y_pred = model_randomforest.predict(X_test)

# print(y_pred)

## 📈 Model Evaluation Performance

We evaluate the performance of the trained Random Forest model using two key regression metrics:

- **RMSE** (Root Mean Squared Error): Measures the average magnitude of prediction errors. Lower values indicate better model performance.

- **R² Score** (Coefficient of Determination): Indicates how well the model explains the variance in the target variable. A value closer to 1.0 means a better fit.


In [94]:
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
r2 = r2_score(y_test, y_pred)

print(f"RMSE for {n_estimators} number of estimators: {rmse:.2f}")
print(f"R² Score for {n_estimators} number of estimators: {r2:.2f}")

RMSE for 100 number of estimators: 20.49
R² Score for 100 number of estimators: 0.88


## 🧪 Track Experiments with MLflow
We use MLflow to track the experiment run, including model parameters, metrics, and the trained model itself:

- Run Name: Automatically generated using the number of estimators (e.g., random_forest_baseline_100) for better traceability.

- Tracking URI: Points to the MLflow server used for logging and managing experiments.

- Experiment Name: Set to "bike_sharing_model" to group related runs.

Within the mlflow.start_run() context:

- We log model parameters like the type (RandomForest) and number of estimators.

- We log evaluation metrics: RMSE and R² score.

- The trained model is saved and logged using mlflow.sklearn.log_model().

This ensures the entire experiment is reproducible and easily comparable within the MLflow dashboard.

In [ ]:
MLFLOW_TRACKING_URI = 'MLFLOW_REMOTE_TRACKING_SERVER'
mlflow.set_tracking_uri(f"{MLFLOW_TRACKING_URI}")
mlflow.set_experiment("bike_sharing_model")

random_num = random.randint(1000, 9999)  # generates a 4-digit random number
run_name = f"random_forest_baseline_{n_estimators}_{random_num}"

print(f"MLflow run name based on the number of estimators: {run_name}")

# directory_path = "../model"
# os.makedirs(directory_path, exist_ok=True)

with mlflow.start_run(run_name=run_name):
    mlflow.log_param("model_type", "RandomForest")
    mlflow.log_param("n_estimators", n_estimators)

    mlflow.log_metric("rmse", rmse)
    mlflow.log_metric("r2", r2)

    mlflow.sklearn.log_model(model_randomforest, "model")
    print("Model and metrics logged to MLflow.")

## 📋 Retrieve and Review Experiment Runs

We query the MLflow tracking server to retrieve all runs associated with the "bike_sharing_model" experiment:

- Use MlflowClient to fetch all runs.

- Extract relevant details such as Run ID, Run Name, RMSE, R², and Start Time.

- Display the runs in a Pandas DataFrame for easier inspection and comparison.

The runs are sorted by start date, allowing us to review recent experiments and identify the best-performing model based on evaluation metrics.

In [92]:
# Get the experiment by name
experiment = mlflow.get_experiment_by_name("bike_sharing_model")

# Load all runs from the experiment
client = mlflow.tracking.MlflowClient()
runs = client.search_runs(experiment_ids=[experiment.experiment_id])

# Display runs in a DataFrame
import pandas as pd

df_runs = pd.DataFrame([{
    "Run ID": run.info.run_id,
    "Run Name": run.data.tags.get("mlflow.runName"),
    "RMSE": run.data.metrics.get("rmse"),
    "R2": run.data.metrics.get("r2"),
    "Date": run.info.start_time
} for run in runs])

df_runs.sort_values("Date", ascending=False).reset_index(drop=True)

,Run ID,Run Name,RMSE,R2,Date
0,4f79bec101d547cba5b11a98b86f5d47,random_forest_baseline_50_2235,20.682602,0.873506,1749575876951
1,218cf2fd397541ce8e3106d998d3e25f,random_forest_baseline_50_5269,20.866728,0.871244,1749575857719
2,753d6da466b54be3af662ecfa40b7011,random_forest_baseline_50_6336,28.849951,0.753878,1749575850003
3,5aeb4210d6414e8d9c63b5f8e561aa36,random_forest_baseline_50_1820,43.473912,0.441122,1749575834377
4,dde1ea36fb894347a1fa7a79b95ff8a6,random_forest_baseline_200_9507,43.458140,0.441527,1749575817326
5,f8c461ccccb443618f8cb058a1c4e0ed,random_forest_baseline_200_2288,28.344997,0.762419,1749575807361
6,de6bbac5ba044e299071796755cd07c7,random_forest_baseline_200_5349,20.734117,0.872875,1749575796513
7,9f41eb6e02ef403cbfedb46d778a15fa,random_forest_baseline_200_7665,20.538956,0.875257,1749575775844
8,0c7d96949a2646e9adf949ea505ecc40,random_forest_baseline_100_7233,20.388602,0.877077,1749575760894
9,337e650547324f86a874712ff97617d5,random_forest_baseline_100_3667,20.491740,0.875830,1749575747423


## 🏆 Select and Register the Best Model Run
The user is prompted to input the run name corresponding to the best-performing experiment. Based on this input:

- We locate the matching run and retrieve its unique run ID.

- Using the run ID, we construct the model URI to register the model in the MLflow Model Registry.

- The model is registered under a descriptive name (BikeSharingModel_{n_estimators}), enabling version control and easy deployment.

This process ensures the chosen model is formally tracked and available for production use.

In [95]:
# Ask user to select a run name
selected_name = input("Enter the run name with the best performance to register its model: ")

# Find the corresponding run ID
selected_run = next(run for run in runs if run.data.tags.get("mlflow.runName") == selected_name)
run_id = selected_run.info.run_id

# Register the model from the selected run
model_uri = f"runs:/{run_id}/model"
result = mlflow.register_model(model_uri, f"BikeSharingModel")

print(f"Model registered: {result.name} v{result.version}")

Enter the run name with the best performance to register its model:  random_forest_baseline_100_7233


Successfully registered model 'BikeSharingModel'.
2025/06/10 17:19:02 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: BikeSharingModel, version 1


Model registered: BikeSharingModel v1


Created version '1' of model 'BikeSharingModel'.
